In [1]:
from julia.api import Julia
jl = Julia(compiled_modules=False)

import julia; julia.install()
from julia import Main

import numpy     as np
import panel     as pn; pn.extension()
import holoviews as hv; hv.extension( "bokeh", logo=False)

from panel.interact import interact

def raster(img):  return hv.Image(img).opts(cmap="gray", xaxis=None, yaxis=None, frame_width=200, aspect='equal')

In [2]:
%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [12]:
%%julia
using LinearAlgebra, LaTeXStrings, Latexify, NearestNeighbors;

<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">The Conjugate Gradients Algorithm</strong>
</div></div>

# 1. The Conjugate Gradient Algorithm

We looked at the objective function
$\qquad y = \frac{1}{2} \left( x_1^2 + b x_2^2\right)\;\;$ for $0 \lt b \le 1$,<br> i.e.,<br>
$\qquad y = x^t A x,\;\;\text{ where } A=\frac{1}{2} \begin{pmatrix} 1 & 0 \\ 0 & b \end{pmatrix}$
in the [Gradient Descent Example notebook](GradientDescentExample.ipynb)

We observed that we could do much better by chhosing a different coordinate system, i.e.,
<br>$\qquad$ setting $A = R^t R,$ where $R = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 0 \\ 0 & \sqrt{b} \end{pmatrix}$ results in $\;F(x) = \tilde{x}_1^2 + \tilde{x}_2^2,\;$ where $\tilde{x} = R x$.

**Idea:**
* Convert $A x = b$ into a minimization problem: $x = argmin_{x} \Vert b - A x \Vert$.
* When $A$ is **positive definite,** it has a Cholesky decomposition $A = R^t R$, so $u^t A u = u^t R^t R u = \Vert R u \Vert^2$:<br>
$\therefore\;\; u^t A u$ is an inner product. The Gram Schmidt procedure generalizes.
* Use a Krylov space: solve for $x$ with the constraint $x \in \mathcal{K}_k$ and search in $A$-orthogonal directions.

$\qquad$ Here is [a nice reference article](https://services.math.duke.edu/~holee/math361-2020/lectures/Conjugate_gradients.pdf)

**Definition:** two non-zero vectors $u$ and $v$ are **conjugate** with respect to a matrix $A$ iff<br>
$\qquad u^t A v = 0.$

**Remarks:**
* if we restrict $A$ to be **symmetric and positive definite**, then $u^t A t$ is an inner product.
* conjugate vectors with respect to a positive definite matrix $A$ are **orthonal with respect to this inner product.**

Start from some vector $r_0$. The natural search direction at the
$k^{th}$ step is the residual $r_k = b - A x_k, k=1,2.\dots$.<br>
Since the residuals $r_0, r_1, r_2, \dots$ are not $A$-orthogonal in general,<br> we will use Gram Schmidt to obtain corresponding $A$-orthogonal vectors $q_0, q_1, q_2, \dots$.

**Theorem:** Let $x_0$ be some vector, and let
$\mathcal{K}_k = \begin{pmatrix} x_0 & A x_0 & \dots & A^{n-1} x_0 \end{pmatrix}$.<br>$\qquad$
The residual $r_k = A x_k - b$ is in $\mathscr{C}\left( \mathcal{K}_k(A,r_0)\right)$

Thus, if $r_k \ne 0,$ the $q_0, q_1, \dots q_k$ forms a basis for $\mathcal{K}_k,$<br>
$\qquad$ and for each $k$, $x_k-x_0 \in \mathcal{K}$.

In [4]:
%%julia
function conjugate_gradients(A, b, x₀, max_iterations=100, tolerance=1e-6)
    n = length(b)
    x = copy(x₀)
    r = b - A * x
    p = copy(r)
    residual_norm_squared = dot(r, r)
    
    for k in 1:max_iterations
        Ap = A * p
        alpha = residual_norm_squared / dot(p, Ap)
        x += alpha * p
        r -= alpha * Ap
        
        new_residual_norm_squared = dot(r, r)
        if sqrt(new_residual_norm_squared) < tolerance
            break
        end
        
        beta = new_residual_norm_squared / residual_norm_squared
        p = r + beta * p
        residual_norm_squared = new_residual_norm_squared
    end
    
    x
end

# Example usage
A = [4.0 -1.0; -1.0 3.0]
b = [1.0, 2.0]
x₀ = [0.0, 0.0]

solution = conjugate_gradients(A, b, x₀)
println("Solution:")
println(solution)

Solution:
[0.4545454545454546, 0.8181818181818182]


In [5]:
%%julia
A\b

array([0.45454545, 0.81818182])

In [8]:
%%julia
function arnoldi_algorithm(A, v, k, tol=1e-6)
    n = size(A, 1)
    Q = zeros(eltype(A), n, k+1)
    H = zeros(eltype(A), k+1, k)

    Q[:, 1] = v / norm(v)

    m = 0
    while m < k
        m += 1
        w = A * Q[:, m]          # Compute the next Krylov vector
        for i in 1:m
            H[i, m] = dot(Q[:, i], w)
            w      -= H[i, m] * Q[:, i]
        end
        H[m+1,m] = norm(w)

        if abs(H[m+1,m]) < tol break end  # we have reached the maximal dimension of the Krylov spaces

        Q[:, m+1] = w / H[m+1, m]
    end

    return Q[:, 1:m], H[1:m, 1:m]
end
function ritz_eigenvalues(A,b,n)
    Q,H = arnoldi_algorithm( A, b, n )
    eigen(H).values
end;

In the following, we generate a random matrix $A \in \mathbb{C}^{200 \times 200}$<br>
and a random starting vector to construct a Krylov matrix,<br>
and we compute the eigenvalues for $\mathcal{K}_3$ as a quick check.

In [9]:
%%julia
N = 200
X = randn(N,N)                  # a real matrix
λ = randn(N) + 1im*randn(N)     # a set of complex eigenvalues
A = X \ diagm(λ) * X            # a matrix with these eigenvalues since X \ Λ  = X⁻¹ Λ
b = randn(N) .+ 0im             # a random real vector used to compute the Krylov matrix
println( L"Ritz eigenvalues for K_3(b)" )
round.(ritz_eigenvalues(A,b,3), digits=3)

$Ritz eigenvalues for K_3(b)$

array([-2.104+0.367j, -0.429+1.471j,  0.805-0.689j])

____
Next, we compute the Ritz eigenvalues and plot them (in red),<br>
together with the exact eigenvalues for the original matrix (in black).

The following display shows the two sets of eigenvalues for successive Krylov space $\mathcal{K}_k$ for $k = 1, 2, \dots n$.

In [10]:
h_exact = hv.Scatter( (np.real(Main.λ), np.imag(Main.λ)), "Re(λ)", "Im(λ)", label="exact" )\
            .opts(size=4, color='black', width=400, height=500, xlim=(-3.5,3.5),ylim=(-3.5,3.5), tools=['hover'])

#def plot_ritz_eigenvals(n):
#    e = Main.ritz_eigenvalues(Main.A, Main.b, n)
#    return (h_exact*hv.Scatter( (np.real(e), np.imag(e)), label="Ritz" ).opts(size=8, title=f"size {n}")).opts(legend_position="top")
#
#interact( plot_ritz_eigenvals, n=(1,Main.N) )

player = pn.widgets.Player(name='Playback', start=1, end=Main.N+1, interval=300)

@pn.depends(value=player.param.value)
def play_ritz_eigenvalues(value):
    e = Main.ritz_eigenvalues(Main.A, Main.b, value)
    return (h_exact*hv.Scatter( (np.real(e), np.imag(e)), label="Ritz" ).opts(size=8, title=f"Exact and Ritz Eigenvalues: size k={value}")).opts(legend_position="top")

pn.Column( player, play_ritz_eigenvalues, width=500)

Column(width=500)
    [0] Player(end=201, interval=300, name='Playback', start=1, value=1, value_throttled=1)
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False)

We observe the **usual behaviour** of the algorithm: **the largest eigenvalues $\mathbf{\vert \lambda \vert}$ converge first.**<br>
when $k = n$, the eigenvalues are identical, and we have obtained all of them.

The algorithm is of great interest since it allows us to stop short of $n$: we frequently are interested in only a number of the largest eigenvalues!

In [13]:
%%julia

# experiment with distance of Ritz eigenvalue to actual eigenvalues
# ------------------------------------------------------------------
# KDTree of actual eigenvalues
ev = eigen(A).values
if isreal(ev[1])
    data   = [ev zeros(size(A,1)) ]'
else
    data   = [real(ev) imag(ev) ]'
end

kdtree = KDTree(data);

# Use the KD Tree to compute the shortest distance of a given point to points in the tree.
#k    = 1
#point = rand(2)
#idxs, dists = knn(kdtree, point, k, true)
#@show point
#@show idxs
#@show data[:,idxs]'
#@show dists;

In [14]:
%%julia
"""
given a set of ritz eigenvalues, compute their distance to the nearest actual eigenvalue
"""
function dist_from_nearest_eval( re, k, kdtree)
    err = []
    for p in sort(re, by = x -> abs(x), rev=true)[1:k]
        i, distances = knn(kdtree, [real(p), imag(p)], 1, true)
        push!(err, distances[1])
    end
    err
end;

In [15]:
%%julia
# compute the distance of the first 6 Ritz eigenvalues to the neares actual eigenvalue
N_eigs=6
errors = []
for k in N_eigs:50+N_eigs
   local d = dist_from_nearest_eval( ritz_eigenvalues(A,b,k), N_eigs, kdtree)
   push!(errors, d)
end

In [16]:
h = \
hv.Overlay( [hv.Curve([f[i] for f in Main.errors], "Iteration Number", "distance", label=f"{i}" ).opts(show_grid=True, logy=True) for i in range(Main.N_eigs)] )\
  .opts(legend_position="top", width=500, height=450, title="Error versus Iteration Number")
h

:Overlay
   .Curve.A_0 :Curve   [Iteration Number]   (distance)
   .Curve.A_1 :Curve   [Iteration Number]   (distance)
   .Curve.A_2 :Curve   [Iteration Number]   (distance)
   .Curve.A_3 :Curve   [Iteration Number]   (distance)
   .Curve.A_4 :Curve   [Iteration Number]   (distance)
   .Curve.A_5 :Curve   [Iteration Number]   (distance)

Let us compare the convergence curve to the errors of the power method to compute a dominant eigenvalue:

In [17]:
%%julia
function power_method( A, x, n)
    errs = []
    x = x / norm(x)
    for i in 1:n
        x_old = x
        x     = A * x
        x     = x / norm( x )
        push!( errs, abs( x'A*x - x_old'A*x_old) ) # change in estimated eigenvalue
    end
    errs
end      
errs = power_method( A, b, 100);

In [18]:
# Overlay the Convergence Curves with the change in the dominant eigenvalue estimate
(h*hv.Curve( Main.errs, label="Power Method" )).opts( width=600, legend_position='right', title='Comparison with Power Method Error')

:Overlay
   .Curve.A_0          :Curve   [Iteration Number]   (distance)
   .Curve.A_1          :Curve   [Iteration Number]   (distance)
   .Curve.A_2          :Curve   [Iteration Number]   (distance)
   .Curve.A_3          :Curve   [Iteration Number]   (distance)
   .Curve.A_4          :Curve   [Iteration Number]   (distance)
   .Curve.A_5          :Curve   [Iteration Number]   (distance)
   .Curve.Power_Method :Curve   [x]   (y)

As we see, the Ritz egenvalue estimates converge faster!

Finally, let us investigate the distances of the Ritz eigenvalues to the closest eigenvalues as a function of $k$.

In [19]:
%%julia
function all_distances( A, b, k, kdtree)
    e = ritz_eigenvalues(A, b, k )
    i, distances = knn(kdtree, [real(e) imag(e)]', 1, true) #'

    if length(i) > 0 ret = [i[1] for i in distances] else ret = [] end
    ret
end
#all_distances(A, b, 4, kdtree);

In [20]:
def histogram_of_distances( A, b, k, kdtree, n_bins=20):
    data = Main.all_distances( A, b, k, kdtree )
    sorted_data = np.sort(data)
    h_spikes = hv.Spikes( (range(len(sorted_data)), sorted_data), "n","distance")\
                 .opts(title="Distances from Eigenvalues", alpha=0.5)
    h_scatter = hv.Scatter(sorted_data).opts(size=2, tools=['hover'], show_grid=True)

    frequencies, edges = np.histogram(data, n_bins)
    h_hist = hv.Histogram((edges, frequencies)).opts(title="Distances Histogram")
    return h_spikes*h_scatter + h_hist

interact( lambda k: histogram_of_distances( Main.A, Main.b, k, Main.kdtree, n_bins=20), k=(1,Main.N))

Column
    [0] Column
        [0] IntSlider(end=200, name='k', start=1, value=100)
    [1] Row
        [0] HoloViews(Layout, name='interactive09449')